<a href="https://colab.research.google.com/github/LMazai/IAe-Podcast/blob/main/friendlyEnglish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ==============================================================================
#  MVP: Chatbot de Ensino de Inglês para Telegram com IA Gemini, Voz e Lembretes
# ==============================================================================
#
# Objetivo:
# Criar um bot funcional (MVP) para Telegram que:
# 1. Responde a mensagens de texto e voz usando a IA Gemini do Google.
# 2. Permite interação e explicações em Inglês e Português.
# 3. Envia mensagens de texto automáticas para incentivar a prática de inglês
#    em horários aleatórios (entre 9h e 19h, fuso de Brasilia),
#    enquanto o bot estiver rodando.
#
# Foco:
# - Clareza do código e funcionalidade MVP.
# - Integração com Telegram, Gemini, STT/TTS.
# - Implementação no Google Colab para aprendizado.

In [2]:
# ==============================================================================
# Passo 1: Instalando Bibliotecas e Configurando API Keys
# ==============================================================================

print("Instalando bibliotecas... Por favor, aguarde.")

# python-telegram-bot v20+ (async)
!pip install -q python-telegram-bot --upgrade
# Para Gemini
!pip install -q google-generativeai
# Para Text-to-Speech (TTS)
!pip install -q pyttsx3 gTTS
# Para Speech-to-Text (STT) e conversão de áudio
!pip install -q SpeechRecognition pydub
# FFmpeg (para pydub e processamento de áudio) e espeak (para pyttsx3 no Linux)
!apt-get update > /dev/null
!apt-get install -y ffmpeg espeak > /dev/null

print("Bibliotecas instaladas!")

# --- Configurando as API Keys (Símbolos Secretos do Colab) ---
import os
from google.colab import userdata
import google.generativeai as genai

# Chave da API do Google para Gemini
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key do Google (Gemini) configurada.")
    API_KEY_CONFIGURED = True
except userdata.SecretNotFoundError:
    print("ERRO: 'GOOGLE_API_KEY' não encontrada nos Símbolos Secretos.")
    API_KEY_CONFIGURED = False
except Exception as e:
    print(f"Erro ao configurar API Key Gemini: {e}")
    API_KEY_CONFIGURED = False

# Token do Bot do Telegram
try:
    TELEGRAM_BOT_TOKEN = userdata.get('TELEGRAM_BOT_TOKEN')
    if not TELEGRAM_BOT_TOKEN: raise userdata.SecretNotFoundError("Token vazio")
    print("Token do Bot do Telegram carregado.")
    BOT_TOKEN_CONFIGURED = True
except userdata.SecretNotFoundError:
    print("ERRO: 'TELEGRAM_BOT_TOKEN' não encontrado ou vazio nos Símbolos Secretos.")
    BOT_TOKEN_CONFIGURED = False
except Exception as e:
    print(f"Erro ao carregar Token do Telegram: {e}")
    BOT_TOKEN_CONFIGURED = False

if not API_KEY_CONFIGURED or not BOT_TOKEN_CONFIGURED:
    print("\nAVISO IMPORTANTE: Configuração de API Key(s) incompleta. O bot pode não funcionar.")

Instalando bibliotecas... Por favor, aguarde.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 58.6 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Bibliotecas instaladas!
API Key do Google (Gemini) configurada.
Token do Bot do Telegram carregado.


In [21]:
# ==============================================================================
# Passo 2: Lógica do Chatbot com Gemini (AJUSTADA PARA CONCISÃO E AUTO-IDIOMA)
# ==============================================================================
import google.generativeai as genai
import asyncio
import re

chat_histories = {} # {chat_id: [historico]}
# chat_language_preferences REMOVIDO

async def get_gemini_response_v3(chat_id: int, user_input_text: str, detected_input_lang: str):
    """
    Gera resposta do Gemini, adaptando-se ao idioma de entrada detectado e focando em concisão.
    Retorna (str: texto_da_resposta_para_usuario, str: texto_tts_se_diferente, str: codigo_idioma_resposta_tts)
    """
    if not API_KEY_CONFIGURED:
        return "My AI brain is not connected (API Key missing).", None, "en"
    if not user_input_text:
        return "I didn't quite get that. Could you rephrase?", None, detected_input_lang

    history = chat_histories.get(chat_id, [])
    response_lang_code_for_tts = detected_input_lang # Resposta no mesmo idioma da entrada

    # --- Preparação do Prompt para Gemini ---
    prompt_parts = []
    tts_text_override = None

    if detected_input_lang == "pt":
        response_lang_code_for_tts = "pt"
        prompt_parts.append(f"""Você é 'English Buddy PT', um tutor de inglês AI amigável e CONCISO no Telegram.
O usuário enviou a seguinte mensagem em PORTUGUÊS: "{user_input_text}"

Sua tarefa (responda SEMPRE em PORTUGUÊS e de forma CURTA e SIMPLES):
1. Se for uma pergunta sobre como dizer algo em inglês (ex: "Como se diz 'X' em inglês?"):
    a. Forneça a tradução direta em inglês.
    b. Se relevante, dê UMA frase de exemplo CURTA em inglês com sua tradução.
    c. EVITE explicações gramaticais longas, a menos que seja crucial para o entendimento da tradução.
2. Se for uma dúvida geral sobre aprendizado: responda de forma CURTA, útil e encorajadora.
3. Se for um comentário: reconheça BREVEMENTE e, se apropriado, ofereça uma forma CURTA de dizer algo similar em inglês.

Exemplo para "Como se diz 'estou com sono' em inglês?":
"Em inglês, 'estou com sono' é 'I'm sleepy'. Por exemplo: 'I'm sleepy, I think I'll go to bed.' (Estou com sono, acho que vou para a cama.)"

Sua resposta (CURTA, SIMPLES, em Português, e com uma explicação em ingles):""")

    else: # detected_input_lang == "en"
        response_lang_code_for_tts = "en"
        prompt_parts.append(f"""You are 'English Buddy', a friendly and CONCISE AI English tutor on Telegram.
The user sent the following message in ENGLISH: "{user_input_text}"

Your task (respond ALWAYS in ENGLISH and keep it SHORT and SIMPLE):
1. Analyze the user's English for critical errors in grammar or vocabulary that hinder understanding.
2. If a CRITICAL error is found:
    a. Provide a corrected version.
    b. VERY BRIEFLY explain the error (1-2 short sentences). Focus on the correction.
    c. Use an encouraging tone. Format: User: "..." / Corrected: "..." / Tip: "..."
3. If the English is understandable or has only minor errors:
    a. Respond naturally and BRIEFLY to continue the conversation.
    b. Avoid unnecessary corrections for minor style issues. Focus on clear communication.
4. If asked for a meaning: provide a SHORT definition and ONE example sentence.

Example for "I has a cat.":
"User: "I has a cat."
Corrected: "I *have* a cat."
Tip: Nice! For 'I', 'you', 'we', 'they', we use 'have'. For 'he', 'she', 'it', we use 'has'."

Your response (SHORT, SIMPLE, in English):""")

    # --- Chamada ao Gemini ---
    try:
        model = genai.GenerativeModel(model_name='gemini-2.0-flash')

        current_turn_history = list(history)
        current_turn_history.append({'role': 'user', 'parts': [user_input_text]}) # Armazenar o input original no histórico

        MAX_HISTORY_TURNS_V3 = 5 # Ainda mais curto para manter o foco
        if len(current_turn_history) > MAX_HISTORY_TURNS_V3 * 2:
            current_turn_history = current_turn_history[-(MAX_HISTORY_TURNS_V3 * 2):]

        # O prompt agora está dentro de `prompt_parts`
        final_prompt_for_gemini = [{'role': 'user', 'parts': prompt_parts}]
        # Adicionar histórico antes do prompt da tarefa atual, se desejado para contexto mais amplo.
        # No entanto, para tarefas de correção/explicação focadas, um prompt direto é melhor.
        # Para manter o contexto da conversa, enviamos o `current_turn_history` que inclui o prompt específico no final.
        # Corrigindo: o prompt específico deve ser a última interação.
        # O histórico é `history`. A interação atual é o `user_input_text` formatado no `prompt_parts`.

        # Opção 1: Histórico + Tarefa específica (pode confundir Gemini se o histórico for em outro idioma)
        # gemini_payload = history + [{'role': 'user', 'parts': prompt_parts}]

        # Opção 2: Focar na tarefa atual, usando o histórico apenas para o Gemini ter um "conhecimento geral"
        # mas a instrução principal é o `prompt_parts`.
        # Para o SDK `generate_content`, ele espera uma lista de turnos.
        # A instrução de sistema é o ideal, mas já estamos colocando no prompt.
        # Vamos enviar o prompt construído como a única mensagem para focar na tarefa.

        print(f"DEBUG Gemini V3 Req: ChatID {chat_id}, InLang {detected_input_lang}, RespLang {response_lang_code_for_tts}, HistLen (para contexto geral, não enviado diretamente) {len(history)}")

        gen_response = await asyncio.to_thread(
            model.generate_content,
            # Aqui, passamos o prompt construído que contém a instrução e o texto do usuário.
            # Se quisermos que o Gemini use o histórico para respostas mais contextuais (não apenas correção/explicação),
            # precisaríamos de uma lógica diferente. Para o MVP atual, focamos na tarefa imediata.
            final_prompt_for_gemini,
            generation_config=genai.types.GenerationConfig(
                candidate_count=1, max_output_tokens=150, # Reduzido para respostas mais curtas
                temperature=0.65 # Um pouco mais factual
            )
        )

        response_text_for_user = gen_response.parts[0].text.strip() if gen_response.parts else \
                                (gen_response.text.strip() if hasattr(gen_response, 'text') and gen_response.text else "")

        if not response_text_for_user:
            return "I had a brief issue generating a response. Could you try again?", None, detected_input_lang

        # Preparar texto para TTS
        if response_lang_code_for_tts == "en" and "Corrected:" in response_text_for_user:
            match = re.search(r"Corrected:\s*\"(.*?)\"", response_text_for_user, re.DOTALL | re.IGNORECASE)
            if match:
                tts_text_override = match.group(1).replace("*", "")
        # Para PT, se a resposta contiver "Em inglês, ... é 'PHRASE'", TTS lê a PHRASE.
        elif response_lang_code_for_tts == "pt":
             match_en_in_pt_explanation = re.search(r"Em inglês, [^']*?\s*é\s*'([^']*)'", response_text_for_user, re.IGNORECASE)
             if match_en_in_pt_explanation:
                 tts_text_override = match_en_in_pt_explanation.group(1)
                 # A resposta de voz para a frase em inglês deve ser em inglês
                 response_lang_code_for_tts = "en" # Sobrescreve o idioma do TTS para a parte em inglês
                 print(f"DEBUG TTS Override: PT explanation, but TTS for English part: '{tts_text_override}' in EN")


        # Atualiza histórico (apenas o input original do usuário e a resposta do bot)
        history.append({'role': 'user', 'parts': [user_input_text]})
        history.append({'role': 'model', 'parts': [response_text_for_user]})
        if len(history) > MAX_HISTORY_TURNS_V3 * 2:
            history = history[-(MAX_HISTORY_TURNS_V3 * 2):]
        chat_histories[chat_id] = history

        return response_text_for_user, tts_text_override, response_lang_code_for_tts

    except Exception as e:
        print(f"Gemini API V3 Error (ChatID {chat_id}): {e}")
        return "I'm having a technical hiccup. Please try again shortly.", None, detected_input_lang

In [17]:
# ==============================================================================
# Passo 3: Funções de Voz (STT e TTS - Sotaque Contextualizado)
# ==============================================================================
# (A função text_to_speech_v2 e speech_to_text_v2 permanecem as mesmas da Célula 4
# da atualização anterior, pois já lidam com sotaques e detecção EN/PT.)
# Apenas para garantir que está aqui:

import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
import os
import asyncio
import time # Assegurar que time está importado

stt_recognizer_v3 = sr.Recognizer() # Nova instância para clareza, se desejado

async def text_to_speech_v3(text: str, chat_id: int, lang_code_and_tld: str = 'en-US', filename_prefix: str = "tts_audio_v3"):
    """
    Converte texto para arquivo de áudio MP3 usando gTTS, com sotaque contextualizado.
    lang_code_and_tld: ex: 'en-US', 'en-GB', 'pt-BR'
    """
    try:
        lang_short = lang_code_and_tld.split('-')[0]
        tld_map = {'en-US': 'com', 'en-GB': 'co.uk', 'en-AU': 'com.au', 'pt-BR': 'com.br'}
        tld = tld_map.get(lang_code_and_tld, 'com' if lang_short == 'en' else 'com.br') # Default TLDs

        unique_id = f"{chat_id}_{lang_short}_{tld.replace('.', '')}_{int(time.time() * 1000)}"
        output_filename = f"{filename_prefix}_{unique_id}.mp3"

        print(f"DEBUG TTS V3: Text='{text[:30]}...', Lang='{lang_short}', TLD='{tld}' for ChatID {chat_id}")
        gtts_obj = gTTS(text=text, lang=lang_short, tld=tld, slow=False)
        await asyncio.to_thread(gtts_obj.save, output_filename)
        return output_filename
    except Exception as e:
        print(f"gTTS V3 Error (ChatID {chat_id}, lang_tld: {lang_code_and_tld}): {e}")
        # Fallback simples se gTTS com TLD falhar
        try:
            lang_short_fb = lang_code_and_tld.split('-')[0]
            fb_filename = f"{filename_prefix}_{chat_id}_{lang_short_fb}_fb_{int(time.time() * 1000)}.mp3"
            gtts_obj_fb = gTTS(text=text, lang=lang_short_fb, slow=False)
            await asyncio.to_thread(gtts_obj_fb.save, fb_filename)
            return fb_filename
        except Exception as e_fb:
            print(f"gTTS V3 Fallback Error: {e_fb}")
            return None


async def speech_to_text_v3(voice_ogg_path: str, chat_id: int):
    """
    Converte áudio OGG para WAV, depois para texto. Tenta EN e PT.
    Retorna (str: texto_reconhecido, str: codigo_idioma_detectado ('en' ou 'pt'))
    """
    base, _ = os.path.splitext(voice_ogg_path)
    temp_wav_path = f"{base}_{chat_id}_temp.wav"
    text_en, text_pt = "", ""

    try:
        audio = await asyncio.to_thread(AudioSegment.from_ogg, voice_ogg_path)
        await asyncio.to_thread(audio.export, temp_wav_path, format="wav")

        with sr.AudioFile(temp_wav_path) as source:
            audio_data = stt_recognizer_v3.record(source) # Usando a instância v3

        # Tenta Inglês
        try:
            text_en = await asyncio.to_thread(stt_recognizer_v3.recognize_google, audio_data, language="en-US")
        except: pass # Silencia erros de reconhecimento individuais
        # Tenta Português
        try:
            text_pt = await asyncio.to_thread(stt_recognizer_v3.recognize_google, audio_data, language="pt-BR")
        except: pass

        print(f"STT V3 Results for ChatID {chat_id}: EN='{text_en[:30]}...', PT='{text_pt[:30]}...'")

        # Lógica de decisão de idioma (prioriza o mais longo se ambos reconhecerem, ou o único reconhecido)
        # Poderia ser mais robusto com análise de confiança, mas não disponível facilmente.
        len_en = len(text_en.split())
        len_pt = len(text_pt.split())

        if len_en > 0 and len_pt == 0: return text_en, "en"
        if len_pt > 0 and len_en == 0: return text_pt, "pt"
        if len_en > 0 and len_pt > 0:
            # Se ambos reconheceram, uma heurística simples: o que tiver mais palavras.
            # Ou, se um for significativamente mais longo.
            if len_pt > len_en + 1: # Se PT for mais que 1 palavra mais longo que EN
                return text_pt, "pt"
            return text_en, "en" # Default para EN se comprimentos próximos ou EN maior

        print(f"STT V3: Nenhum idioma reconhecido claramente para ChatID {chat_id}")
        return "", None

    except Exception as e:
        print(f"STT V3 Error (ChatID {chat_id}, File: {voice_ogg_path}): {e}")
        return "Error during speech recognition.", None
    finally:
        if os.path.exists(temp_wav_path): os.remove(temp_wav_path)

In [18]:
# ==============================================================================
# Passo 4: Código Principal do Bot do Telegram (SIMPLIFICADO, SEM COMANDOS DE IDIOMA)
# ==============================================================================
from telegram import Update, ForceReply # ReplyKeyboardMarkup, KeyboardButton REMOVIDOS
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
from telegram.constants import ParseMode
import random
import datetime
import pytz
import os
import asyncio
# import time # Já importado em células anteriores

# --- Armazenamento de IDs e Mensagens Motivacionais (como antes) ---
schedulable_chat_ids_v3 = set()
reminder_messages_audio_en_v3 = [
    "Hey! Quick English boost: How would you say 'Estou animado para o fim de semana' in English?",
    "Practice time! Try to describe your favorite food to me in English. 🎤",
    "A little English a day keeps the language barrier away! What's up?",
    "Let's make today an English day! Send a voice note about your plans."
] # Mensagens mais curtas

# --- Tarefa Agendada para Envio de Lembretes em ÁUDIO (como antes, mas usando text_to_speech_v3) ---
async def scheduled_audio_reminder_task_v3(application: Application):
    # (Mesma lógica da função scheduled_audio_reminder_task da atualização anterior,
    # mas chamará text_to_speech_v3)
    try:
        TARGET_TIMEZONE_V3 = pytz.timezone('America/Sao_Paulo')
    except pytz.exceptions.UnknownTimeZoneError:
        TARGET_TIMEZONE_V3 = pytz.utc
    print(f"INFO: Tarefa de lembretes em ÁUDIO V3 iniciada (fuso: {TARGET_TIMEZONE_V3}).")
    while True:
        now_local = datetime.datetime.now(TARGET_TIMEZONE_V3)
        if 9 <= now_local.hour <= 19:
            await asyncio.sleep(random.randint(75 * 60, 180 * 60)) # Intervalo 1.15h a 3h
            current_local_after_sleep = datetime.datetime.now(TARGET_TIMEZONE_V3)
            if not (9 <= current_local_after_sleep.hour <= 19): continue

            if random.random() < 0.15 and schedulable_chat_ids_v3: # Chance menor (15%)
                target_chat_id = random.choice(list(schedulable_chat_ids_v3))
                message_text_for_tts = random.choice(reminder_messages_audio_en_v3)
                tts_file = await text_to_speech_v3(message_text_for_tts, target_chat_id, lang_code_and_tld='en-US', filename_prefix="reminder_audio_v3")
                if tts_file:
                    try:
                        print(f"SCHEDULER V3 (Audio): Sending reminder to ChatID {target_chat_id} at {current_local_after_sleep.strftime('%H:%M')}")
                        await application.bot.send_voice(chat_id=target_chat_id, voice=open(tts_file, 'rb'))
                    except Exception as e:
                        print(f"SCHEDULER V3 (Audio) Error: {e}")
                        if "chat not found" in str(e).lower() or "bot was blocked" in str(e).lower():
                            schedulable_chat_ids_v3.discard(target_chat_id)
                    finally:
                        if os.path.exists(tts_file): os.remove(tts_file)
        else:
            next_run_time = now_local.replace(hour=9, minute=0, second=0, microsecond=0)
            if now_local.hour > 19 : next_run_time += datetime.timedelta(days=1)
            sleep_duration_seconds = (next_run_time - now_local).total_seconds()
            if sleep_duration_seconds <= 0:
                next_run_time += datetime.timedelta(days=1)
                sleep_duration_seconds = (next_run_time - now_local).total_seconds()
                if sleep_duration_seconds <= 0: sleep_duration_seconds = 60 * 60
            print(f"SCHEDULER V3 (Audio): Fora do horário. Dormindo por {sleep_duration_seconds/3600:.1f}h.")
            await asyncio.sleep(max(60, sleep_duration_seconds))


# --- Handlers de Comando e Mensagem (Simplificados) ---
async def start_handler_v3(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    chat_id = update.effective_chat.id
    schedulable_chat_ids_v3.add(chat_id)
    # chat_language_preferences REMOVIDO do start

    welcome_msg = (
        f"Hello {user.mention_html()}! I'm <b>English Buddy</b> 🤖, your AI practice partner.\n\n"
        "I'll automatically detect if you're speaking 🇬🇧 English or 🇧🇷 Portuguese and respond in the same language.\n"
        "My goal is to help you practice! If you write/speak in English, I'll offer corrections. If you ask in Portuguese how to say something, I'll explain and translate.\n\n"
        "Just send me ✍️ text or 🎙️ voice messages to begin.\n"
        "I'll also send short motivational voice notes in English sometimes (9am-7pm São Paulo time).\n\n"
        "Type /help for a quick guide."
    )
    # ReplyKeyboardMarkup REMOVIDO
    await update.message.reply_html(welcome_msg) # reply_markup REMOVIDO

    if chat_id in chat_histories: del chat_histories[chat_id] # Reseta histórico
    print(f"CMD /start V3: User {user.id} (ChatID {chat_id}). Auto-language. Added to schedulable list.")

async def help_handler_v3(update: Update, context: ContextTypes.DEFAULT_TYPE):
    help_msg = (
        "<b>English Buddy - How I Work</b> 🧐\n\n"
        "<code>/start</code> - Initialize or reset our chat.\n"
        "<code>/help</code> - Show this guide.\n\n"
        "🗣️ **Automatic Language Detection:**\n"
        "  - If you message in English, I'll reply in English (and correct if needed).\n"
        "  - If you message in Portuguese, I'll reply in Portuguese (and translate/explain English phrases if you ask).\n\n"
        "➡️ **Input = Output Type:**\n"
        "  - Text message in ➔ Text message out.\n"
        "  - Voice message in ➔ Voice message out.\n\n"
        "Just start chatting naturally! My replies aim to be short and clear."
    )
    await update.message.reply_html(help_msg)

# Comando /lang REMOVIDO

# Função unificada para processar interação (adaptada)
async def handle_user_interaction_v3(update: Update, context: ContextTypes.DEFAULT_TYPE, user_input_text: str, detected_input_lang: str, is_voice_input: bool):
    chat_id = update.effective_chat.id
    schedulable_chat_ids_v3.add(chat_id)

    action = "record_voice" if is_voice_input else "typing"
    await context.bot.send_chat_action(chat_id=chat_id, action=action)

    response_text_for_user, tts_text_override, response_lang_code_for_tts = await get_gemini_response_v3(chat_id, user_input_text, detected_input_lang)

    if is_voice_input:
        text_for_tts = tts_text_override if tts_text_override else response_text_for_user
        sotaque_tts = "pt-BR" if response_lang_code_for_tts == "pt" else "en-US" # Default sotaque EN para US

        print(f"DEBUG V3 Voice Output: TTS Lang '{response_lang_code_for_tts}', Sotaque '{sotaque_tts}', Text='{text_for_tts[:30]}...'")
        if text_for_tts and "API Key missing" not in text_for_tts and "error" not in text_for_tts.lower():
            tts_file = await text_to_speech_v3(text_for_tts, chat_id, lang_code_and_tld=sotaque_tts)
            if tts_file:
                try:
                    await context.bot.send_voice(chat_id=chat_id, voice=open(tts_file, 'rb'))
                except Exception as e_send_voice:
                    print(f"Error sending V3 TTS voice (ChatID {chat_id}): {e_send_voice}")
                    await update.message.reply_text(f"(Audio failed, text reply instead):\n{response_text_for_user}")
                finally:
                    if os.path.exists(tts_file): os.remove(tts_file)
            else:
                await update.message.reply_text(f"(TTS generation failed, text reply instead):\n{response_text_for_user}")
        else:
             await update.message.reply_text(response_text_for_user)
    else: # Input texto -> Output texto
        # Aqui, poderíamos usar ParseMode.HTML se Gemini for instruído a usar <b> para correções.
        # Ex: response_text_for_user = "Corrected: I <b>went</b> to the store."
        # await update.message.reply_html(response_text_for_user)
        # Por ora, mantendo simples. O Gemini já tem "Corrected:" etc.
        await update.message.reply_text(response_text_for_user)


async def text_message_handler_v3(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    user_text = update.message.text
    print(f"TEXT V3 from ChatID {chat_id}: '{user_text[:50]}...'")

    # Heurística simples para detecção de idioma no texto (pode ser melhorada com langdetect)
    # Conta palavras-chave simples em PT. Se presentes, assume PT, senão EN.
    pt_keywords = [" o que ", " que ", " como ", " se ", " em ", " para ", " de ", " com ", " um ", " uma ", " eu ", " você "]
    en_keywords = [" the ", " is ", " are ", " to ", " and ", " a ", " an ", " in ", " for ", " I ", " you "]

    pt_score = sum(1 for kw in pt_keywords if kw in user_text.lower())
    en_score = sum(1 for kw in en_keywords if kw in user_text.lower())

    # Pequena preferência para inglês se os scores forem próximos, para incentivar a prática.
    detected_lang = "pt" if pt_score > en_score + 1 else "en"

    # Se a frase for muito curta, pode ser difícil detectar, default para 'en'
    if len(user_text.split()) < 3 and pt_score == 0 and en_score == 0:
        detected_lang = 'en' # Ou poderia ser o último idioma usado, se rastreado.

    print(f"DEBUG Text V3 Handler: Inferred Input Lang for Gemini '{detected_lang}' (PT Score: {pt_score}, EN Score: {en_score})")
    await handle_user_interaction_v3(update, context, user_text, detected_lang, is_voice_input=False)

async def voice_message_handler_v3(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    voice = update.message.voice
    print(f"VOICE V3 from ChatID {chat_id}, Duration: {voice.duration}s")

    file_id = voice.file_id
    voice_file_obj = await context.bot.get_file(file_id)
    temp_ogg_path = f"voice_input_v3_{chat_id}_{int(time.time())}.ogg"
    await voice_file_obj.download_to_drive(temp_ogg_path)

    recognized_text, detected_lang_code = await speech_to_text_v3(temp_ogg_path, chat_id)
    if os.path.exists(temp_ogg_path): os.remove(temp_ogg_path)

    if not recognized_text or detected_lang_code is None:
        reply_err_text = "Sorry, I couldn't quite make out what you said. Could you try speaking a bit more clearly?"
        await update.message.reply_text(reply_err_text)
        sotaque_tts_err = "en-US" # Default para erro
        if detected_lang_code == "pt": sotaque_tts_err = "pt-BR" # Se STT retornou algo, mesmo que vazio
        tts_err_file = await text_to_speech_v3(reply_err_text, chat_id, lang_code_and_tld=sotaque_tts_err)
        if tts_err_file:
            try: await context.bot.send_voice(chat_id=chat_id, voice=open(tts_err_file, 'rb'))
            finally:
                if os.path.exists(tts_err_file): os.remove(tts_err_file)
        return

    # Não envia mais o "Heard: ..." para ser mais direto.
    # await update.message.reply_text(f"Heard (in {detected_lang_code}): \"<i>{recognized_text}</i>\"", parse_mode=ParseMode.HTML)

    await handle_user_interaction_v3(update, context, recognized_text, detected_lang_code, is_voice_input=True)

# --- Função Principal do Bot (V3) ---
async def run_bot_v3():
    if not BOT_TOKEN_CONFIGURED or not API_KEY_CONFIGURED:
        print("FATAL: Bot token or Gemini API key not configured. Exiting.")
        return

    app = Application.builder().token(TELEGRAM_BOT_TOKEN).build()

    app.add_handler(CommandHandler("start", start_handler_v3))
    app.add_handler(CommandHandler("help", help_handler_v3))
    # Comando /lang REMOVIDO
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, text_message_handler_v3))
    app.add_handler(MessageHandler(filters.VOICE, voice_message_handler_v3))

    print("INFO: Telegram Bot V3 (Auto-Lang, Concise) starting...")
    try:
        await app.initialize()
        asyncio.create_task(scheduled_audio_reminder_task_v3(app)) # Agendador de áudio
        await app.updater.start_polling()
        await app.start()
        print("INFO: Bot V3 is running. Press Ctrl+C (or interrupt Colab cell) to stop.")
        while True: await asyncio.sleep(3600)
    except (KeyboardInterrupt, SystemExit): print("INFO: Bot V3 shutdown initiated.")
    except Exception as e: print(f"CRITICAL: Unhandled error in Bot V3 main loop: {e}")
    finally:
        print("INFO: Shutting down Bot V3 application...")
        if hasattr(app, 'updater') and app.updater and app.updater.running: await app.updater.stop()
        # if hasattr(app, 'running') and app.running: await app.stop() # app.stop() não é mais o método para PTB v20+
        if hasattr(app, 'stop') and callable(getattr(app, 'stop')): # Checa se o método stop existe e é chamável
            await app.stop()
        if hasattr(app, 'shutdown'): await app.shutdown()
        print("INFO: Bot V3 application shut down.")
        await cleanup_temp_files_v3() # Função de limpeza

# Função de limpeza (renomeada para V3, mas mesma lógica)
async def cleanup_temp_files_v3(directory=".", age_seconds=0):
    # (Mesmo código da função cleanup_temp_files da atualização anterior)
    count = 0
    if directory == ".": directory = "/content/"
    print(f"INFO: Cleaning up V3 temporary files in {directory}...")
    for filename in os.listdir(directory):
        if (filename.startswith("tts_audio_v3_") and filename.endswith(".mp3")) or \
           (filename.startswith("reminder_audio_v3_") and filename.endswith(".mp3")) or \
           (filename.startswith("voice_input_v3_") and filename.endswith(".ogg")): # Adicionado .ogg
            try:
                full_path = os.path.join(directory, filename)
                os.remove(full_path)
                count += 1
            except Exception as e: print(f"Cleanup V3 Error deleting {filename}: {e}")
    if count > 0: print(f"INFO Cleanup V3: Deleted {count} temporary audio files.")
    else: print("INFO Cleanup V3: No temporary audio files to delete.")

In [12]:
# Função de limpeza (copiada da versão anterior, pois é a mesma)
async def cleanup_temp_files(directory=".", age_seconds=0): # Deleta todos os .mp3 e .ogg temporários
    count = 0
    # Garantir que estamos no diretório correto, especialmente no Colab
    # O diretório padrão do Colab é /content/
    if directory == ".":
        directory = "/content/"

    print(f"INFO: Cleaning up temporary files in {directory}...")
    for filename in os.listdir(directory):
        if (filename.startswith("tts_audio_") and filename.endswith(".mp3")) or \
           (filename.startswith("reminder_audio_") and filename.endswith(".mp3")) or \
           (filename.startswith("voice_input_") and filename.endswith(".ogg")):
            try:
                full_path = os.path.join(directory, filename)
                os.remove(full_path)
                # print(f"DEBUG Cleanup: Deleted {full_path}") # Muito verboso
                count += 1
            except Exception as e:
                print(f"Cleanup Error deleting {filename}: {e}")
    if count > 0: print(f"INFO Cleanup: Deleted {count} temporary audio files.")
    else: print("INFO Cleanup: No temporary audio files to delete in this session.")

In [23]:
# ==============================================================================
# Passo 5: Executando o Bot do Telegram (Versão 3)
# ==============================================================================
import asyncio
import nest_asyncio

nest_asyncio.apply()

if API_KEY_CONFIGURED and BOT_TOKEN_CONFIGURED:
    print("INFO: Configuration V3 check OK. Starting Telegram bot V3 execution...")
    try:
        asyncio.run(run_bot_v3()) # Chama a nova função principal V3
    except KeyboardInterrupt:
        print("\nINFO: Bot V3 execution interrupted by user.")
    except Exception as e:
        print(f"CRITICAL: Error running the Bot V3 cell: {e}")
    finally:
        print("INFO: Bot V3 execution cell finished.")
        # A limpeza agora é chamada dentro do `finally` de `run_bot_v3`
else:
    print("ERROR: Bot V3 cannot start due to missing API Key or Bot Token. Check Cell 2 and Colab Secrets.")

INFO: Configuration V3 check OK. Starting Telegram bot V3 execution...
INFO: Telegram Bot V3 (Auto-Lang, Concise) starting...
INFO: Tarefa de lembretes em ÁUDIO V3 iniciada (fuso: America/Sao_Paulo).
SCHEDULER V3 (Audio): Fora do horário. Dormindo por 12.2h.
INFO: Bot V3 is running. Press Ctrl+C (or interrupt Colab cell) to stop.
TEXT V3 from ChatID 7548144916: 'Como se diz eu te amo...'
DEBUG Text V3 Handler: Inferred Input Lang for Gemini 'pt' (PT Score: 2, EN Score: 0)
DEBUG Gemini V3 Req: ChatID 7548144916, InLang pt, RespLang pt, HistLen (para contexto geral, não enviado diretamente) 6
VOICE V3 from ChatID 7548144916, Duration: 2s
STT V3 Results for ChatID 7548144916: EN='como se dice El Chicano...', PT='como se diz eu te amo em inglê...'
DEBUG Gemini V3 Req: ChatID 7548144916, InLang pt, RespLang pt, HistLen (para contexto geral, não enviado diretamente) 8
DEBUG V3 Voice Output: TTS Lang 'pt', Sotaque 'pt-BR', Text='"Eu te amo" em inglês é "I lov...'
DEBUG TTS V3: Text='"Eu te am